In [ ]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
)
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler
from sklearn.svm import SVC

import prepare_data
from components import PCOA
from helpers import PipelineHelper
import settings

In [ ]:
# What happened so far: DB extract and blank procedure. Now import resulting MP data from csv
mp_pdd = prepare_data.get_pdd()

# Also import sediment data (sediment frequencies per size bin from master sizer export)
grainsize_iow, grainsize_cau = prepare_data.get_grainsizes()[0:2]
scor_iow = PCOA(grainsize_iow, 2)[0]
scor_cau = PCOA(grainsize_cau, 2)[0]

# ...some data wrangling to prepare particle domain data and sample domain data for MP and combine with certain sediment aggregates.
sdd_iow = prepare_data.aggregate_SDD(mp_pdd)
sdd_iow = prepare_data.additional_sdd_merging(sdd_iow, how='outer')
sdd_iow = sdd_iow.merge(scor_iow, right_index=True, left_on='Sample', how='outer')

sdd_cau = pd.read_csv('../data/Metadata_CAU_sampling_log.csv', index_col=0).join(pd.read_csv('../data/GRADISTAT_CAU_vol_log-cau_closed.csv', index_col=0), how='outer')
sdd_cau = sdd_cau.merge(scor_cau, right_index=True, left_on='Sample', how='outer').reset_index()

In [ ]:
sdd_iow = sdd_iow.replace({'Sample': settings.shortnames}).sort_values(by='Sample')
model_data = sdd_iow.loc[~sdd_iow.Concentration.isna()].set_index('Sample')
pred_data = sdd_iow.loc[sdd_iow.Concentration.isna()]
pred_data = pd.concat([pred_data, sdd_cau.drop('Date',axis=1)]).set_index('Sample')

In [ ]:
model_data.to_csv('../data/model_data.csv')
pred_data.to_csv('../data/pred_data.csv')

In [ ]:
dp = round(pd.read_csv('../data/Calc_depth_pred.csv', index_col=0)[['Calc_depth_1']]*-1,2)
toc = pd.read_csv('../data/GIS_Greiferproben SHS_ETRS89_UTMZ32N.csv', index_col=1)[['Corg [%]']]
meta = pd.read_csv('../data/Metadata_CAU_sampling_log.csv', index_col=0)

In [ ]:
meta.join(dp).drop('Depth',axis=1).join(toc).rename(columns={'Corg [%]':'TOC', 'Calc_depth_1': 'Depth'}).to_csv('../data/Metadata_CAU_sampling_log.csv')

In [ ]:
targetlist = ['Concentration']

featurelist = [
    'Depth',
    'LON', 'LAT',
    # 'Dist_Marina', 'Dist_WWTP', 'Dist_WWTP2',
    'MODE 1 (µm)',
    'D10 (µm)', 'D50 (µm)', 'D90 (µm)',
    'perc GRAVEL', 'perc SAND', 'perc MUD', 'perc CLAY',
    #'OM_D50', 'TOC', 'Hg', 'TIC', 'regio_sep',
    'PC1', 'PC2'
    ]
model_X = model_data[featurelist]
model_y = model_data[targetlist]
pred_X = pred_data[featurelist]

In [ ]:

pipe = Pipeline([
    ('scaler', PipelineHelper([
        ('std', StandardScaler()),
        ('max', MaxAbsScaler()),
    ], optional=True)),

    ('classifier', PipelineHelper([
        # ('svm', SVC()),
        # ('rf', RandomForestClassifier()),
        # ('ada', AdaBoostClassifier()),
        # ('gb', GradientBoostingClassifier()),
        ('knn', KNeighborsClassifier()),
        # ('nb_pipe', Pipeline([
        #     # Naivie Bayes needs positive numbers
        #     ('scaler', MinMaxScaler()),
        #     ('nb', MultinomialNB()),
        # ])),
    ])),
])

params = {
    'scaler__selected_model': pipe.named_steps['scaler'].generate(
        {
            'std__with_mean': [True, False],
            'std__with_std': [True, False],
            # no params for 'max' leads to using standard params
        }
    ),
    'classifier__selected_model': pipe.named_steps['classifier'].generate(
        {
            # 'svm__C': [0.1, 1.0],
            # 'svm__kernel': ['linear', 'rbf'],
            # 'rf__n_estimators': [10, 20, 50, 100, 150],
            # 'rf__max_features': ['sqrt', 'log2'],
            # 'rf__min_samples_split': [2, 5, 10],
            # 'rf__min_samples_leaf': [1, 2, 4],
            # 'rf__bootstrap': [True, False],
            # 'ada__n_estimators': [10, 20, 40, 100],
            # 'ada__algorithm': ['SAMME', 'SAMME.R'],
            # 'gb__n_estimators': [10, 20, 50, 100],
            # 'gb__criterion': ['friedman_mse', 'squared_error'],
            # 'gb__max_features': ['sqrt', None],
            'knn__n_neighbors': [2, 3, 5, 7, 10],
            'knn__leaf_size': [1, 2, 3, 5],
            'knn__weights': ['uniform', 'distance'],
            'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            # 'nb_pipe__nb__fit_prior': [True, False],
            # 'nb_pipe__nb__alpha': [0.1, 0.2],
        }
    ),
}


In [ ]:
grid = GridSearchCV(pipe, params, scoring='accuracy', verbose=1, n_jobs=-1)
grid.fit(X, y)


In [ ]:
print(grid.best_params_)
print(grid.best_score_)